In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import pickle
import utils
import moran_auto
import gmm
import kmeans_utils
import figure_functions
from sklearn.preprocessing import MinMaxScaler
from sklearn import mixture
from sklearn.cluster import KMeans
import csv
from scipy.misc import imread
from collections import defaultdict
from matplotlib.patches import Ellipse
from matplotlib.patches import Polygon

In [2]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data/'
fig_path = curr_dir + '/../figs/'
results_path = curr_dir + '/../results'
animation_path = curr_dir + '/../animation'

In [3]:
park_data_new, gps_loc, N = utils.load_daily_data_standalone(data_path)

In [4]:
results = gmm.locational_demand_analysis(park_data_new, gps_loc, N)

days = [result[0] for result in results]
hours = [result[1] for result in results]
errors = [result[2] for result in results]
morans_mixture = [result[3] for result in results]
morans_adjacent = [result[4] for result in results]
means = [result[5] for result in results]

In [20]:
distances, centroids = kmeans_utils.get_distances(means)

In [6]:
all_time_points = kmeans_utils.get_centroid_circle_paths(distances, centroids)

In [7]:
fig, ax = figure_functions.centroid_radius(centroids, all_time_points, gps_loc, times=range(6), 
                                               fig_path=fig_path, shape=(2,3))

# Train and Test GMM

In [ ]:
park_data, gps_loc, N = utils.load_daily_data(data_path)

In [ ]:
check = np.array(sorted(park_data['Load'].values.tolist(), reverse=True))

In [ ]:
len(np.where(check > 1.5)[0])/float(len(check)) * 100

In [ ]:
pd.DataFrame(range(10), index=range(10), columns=['check'])

In [ ]:
results = gmm.locational_demand_analysis(park_data, gps_loc, N)

In [ ]:
days = [result[0] for result in results]
hours = [result[1] for result in results]
errors = [result[2] for result in results]
morans_mix = [result[3] for result in results]
morans_adj = [result[4] for result in results]
means = [result[5] for result in results]

In [ ]:
scores, times = kmeans_utils.get_time_scores(means)

In [ ]:
best_times = {}
best_days = {}
for time in times:
    day = time/12
    hour = time%12 + 8
    if hour not in best_times:
        best_times[hour] = time
    if day not in best_days and hour not in [8, 15,16,17]:
        print day, hour
        best_days[day] = time

In [ ]:
good_times = []

for time in times:
    if time % 12 + 8 in [8,9]:
        continue
        
    good_times.append(time)
    data = np.vstack((means[time]))
    kmeans = KMeans(n_clusters=3).fit(data)
    labels = kmeans.labels_.tolist()

In [ ]:
good_times = best_days.values()

In [ ]:
fig, ax = figure_functions.centroid_plots(means, gps_loc, N, times=good_times, fig_path=fig_path, shape=(2,3))